# Car Price Prediction: Data Preprocessing / EDA
## Project Members: 
### 1.    Abhishek Manish Bedarkar | 50495186 | abedarka
### 2.    Sanket Milind Gadhave | 50495154 | sgadhave
### 3.    Milind Rajendra Kulkarni | 50495311 | milindra

In [56]:
# import all required libraries 
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import r2_score
from scipy import stats
import xgboost as xgb
from sklearn.neighbors import KNeighborsRegressor
from pandas.plotting import scatter_matrix
from joblib import dump

In [57]:
# Read csv file and render top 10 rows
df = pd.read_csv("car_price_prediction.csv")
df.head(10).T

,0,1,2,3,4,5,6,7,8,9
ID,45654403,44731507,45774419,45769185,45809263,45802912,45656768,45816158,45641395,45756839
Price,13328,16621,8467,3607,11726,39493,1803,549,1098,26657
Levy,1399,1018,-,862,446,891,761,751,394,-
Manufacturer,LEXUS,CHEVROLET,HONDA,FORD,HONDA,HYUNDAI,TOYOTA,HYUNDAI,TOYOTA,LEXUS
Model,RX 450,Equinox,FIT,Escape,FIT,Santa FE,Prius,Sonata,Camry,RX 350
Prod. year,2010,2011,2006,2011,2014,2016,2010,2013,2014,2007
Category,Jeep,Jeep,Hatchback,Jeep,Hatchback,Jeep,Hatchback,Sedan,Sedan,Jeep
Leather interior,Yes,No,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes
Fuel type,Hybrid,Petrol,Petrol,Hybrid,Petrol,Diesel,Hybrid,Petrol,Hybrid,Petrol
Engine volume,3.5,3,1.3,2.5,1.3,2,1.8,2.4,2.5,3.5


In [58]:
# List of features in dataset
df.columns.to_list()

['ID',
 'Price',
 'Levy',
 'Manufacturer',
 'Model',
 'Prod. year',
 'Category',
 'Leather interior',
 'Fuel type',
 'Engine volume',
 'Mileage',
 'Cylinders',
 'Gear box type',
 'Drive wheels',
 'Doors',
 'Wheel',
 'Color',
 'Airbags']

In [59]:
# Print summary of dataset and have a look at central tendancy
df.describe().T

,count,mean,std,min,25%,50%,75%,max
ID,19237.0,4.557654e+07,936591.422799,20746880.0,45698374.0,45772308.0,45802036.0,45816654.0
Price,19237.0,1.855593e+04,190581.269684,1.0,5331.0,13172.0,22075.0,26307500.0
Prod. year,19237.0,2.010913e+03,5.668673,1939.0,2009.0,2012.0,2015.0,2020.0
Cylinders,19237.0,4.582991e+00,1.199933,1.0,4.0,4.0,4.0,16.0
Airbags,19237.0,6.582627e+00,4.320168,0.0,4.0,6.0,12.0,16.0


# Data Preprocessing
## 1. Column Renaming

In [60]:
# Renaming dataset attributes for better understanding
df.rename(columns={'Prod. year': 'Prod_year'}, inplace=True)
df.rename(columns={'Leather interior': 'Leather_interior'}, inplace=True)
df.rename(columns={'Fuel type': 'Fuel_type'}, inplace=True)
df.rename(columns={'Engine volume': 'Engine_volume'}, inplace=True)
df.rename(columns={'Gear box type': 'Gear_box_type'}, inplace=True)
df.rename(columns={'Drive wheels': 'Drive_wheels'}, inplace=True)

## 2. Removing Duplicates

In [61]:
# Dimention of dataset before removing duplicates 
df.shape

(19237, 18)

In [62]:
# Count the number of duplicates in dataset
df.duplicated().sum()

313

In [63]:
# Drop duplicate rows from dataset
df = df.drop_duplicates()

In [64]:
# Dimention of dataset after removing duplicates
df.shape

(18924, 18)

## 3. Handling inconsistent Mileage values

In [65]:
# remove km from mileage

df["Mileage"] = df["Mileage"].str.replace(' km', '')  # Replace spaces with underscores

df.head(5)

,ID,Price,Levy,Manufacturer,Model,Prod_year,Category,Leather_interior,Fuel_type,Engine_volume,Mileage,Cylinders,Gear_box_type,Drive_wheels,Doors,Wheel,Color,Airbags
0,45654403,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
1,44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8
2,45774419,8467,-,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000,4.0,Variator,Front,04-May,Right-hand drive,Black,2
3,45769185,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966,4.0,Automatic,4x4,04-May,Left wheel,White,0
4,45809263,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901,4.0,Automatic,Front,04-May,Left wheel,Silver,4


## 4. Handle incorrect format of Levy

In [66]:
# Replace '-' and '' with '0'
df["Levy"] = df["Levy"].str.replace("-", '')
df["Levy"] = df["Levy"].str.replace("", '0')


## 5. Feature Engineering

In [67]:
# Create new enriched column 'With_Turbo' and remove 'Turbo' from eng
df['With_Turbo'] = df['Engine_volume'].str.contains(' Turbo', case=False).astype(int)
df["Engine_volume"] = df["Engine_volume"].str.replace(' Turbo', '')
df.head()

,ID,Price,Levy,Manufacturer,Model,Prod_year,Category,Leather_interior,Fuel_type,Engine_volume,Mileage,Cylinders,Gear_box_type,Drive_wheels,Doors,Wheel,Color,Airbags,With_Turbo
0,45654403,13328,010309090,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005,6.0,Automatic,4x4,04-May,Left wheel,Silver,12,0
1,44731507,16621,010001080,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8,0
2,45774419,8467,0,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000,4.0,Variator,Front,04-May,Right-hand drive,Black,2,0
3,45769185,3607,0806020,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966,4.0,Automatic,4x4,04-May,Left wheel,White,0,0
4,45809263,11726,0404060,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901,4.0,Automatic,Front,04-May,Left wheel,Silver,4,0


## 6. Handling Null Values

In [68]:
# Check for any missing values
df.isnull().sum()
# Since there are no missing values we don't have to worry about filling 
# missing values

ID                  0
Price               0
Levy                0
Manufacturer        0
Model               0
Prod_year           0
Category            0
Leather_interior    0
Fuel_type           0
Engine_volume       0
Mileage             0
Cylinders           0
Gear_box_type       0
Drive_wheels        0
Doors               0
Wheel               0
Color               0
Airbags             0
With_Turbo          0
dtype: int64

## 7. Datatype conversion

In [69]:
# Converting datatypes of attributes 
df['Manufacturer'] = df['Manufacturer'].astype('category')
df['Category'] = df['Category'].astype('category')
df['Leather_interior'] = df['Leather_interior'].astype('category')
df['Fuel_type'] = df['Fuel_type'].astype('category')
df['Gear_box_type'] = df['Gear_box_type'].astype('category')
df['Drive_wheels'] = df['Drive_wheels'].astype('category')
df['Doors'] = df['Doors'].astype('category')
df['Wheel'] = df['Wheel'].astype('category')
df['Color'] = df['Color'].astype('category')
df['Model'] = df['Model'].astype('category')
df["Mileage"] = df["Mileage"].astype(str).astype(int)
df["Levy"] = df["Levy"].astype(str).astype(float)
df["Engine_volume"] = df["Engine_volume"].astype(str).astype(float)

In [70]:
# To render the change in datatype conversion
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18924 entries, 0 to 19236
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   ID                18924 non-null  int64   
 1   Price             18924 non-null  int64   
 2   Levy              18924 non-null  float64 
 3   Manufacturer      18924 non-null  category
 4   Model             18924 non-null  category
 5   Prod_year         18924 non-null  int64   
 6   Category          18924 non-null  category
 7   Leather_interior  18924 non-null  category
 8   Fuel_type         18924 non-null  category
 9   Engine_volume     18924 non-null  float64 
 10  Mileage           18924 non-null  int32   
 11  Cylinders         18924 non-null  float64 
 12  Gear_box_type     18924 non-null  category
 13  Drive_wheels      18924 non-null  category
 14  Doors             18924 non-null  category
 15  Wheel             18924 non-null  category
 16  Color             18924 non

## 8. Drop irrelevant columns

In [71]:
# Drop ID column as it wont contribute in prediction
df.drop(columns='ID', inplace=True)
df.head(5)

,Price,Levy,Manufacturer,Model,Prod_year,Category,Leather_interior,Fuel_type,Engine_volume,Mileage,Cylinders,Gear_box_type,Drive_wheels,Doors,Wheel,Color,Airbags,With_Turbo
0,13328,10309090.0,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005,6.0,Automatic,4x4,04-May,Left wheel,Silver,12,0
1,16621,10001080.0,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3.0,192000,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8,0
2,8467,0.0,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000,4.0,Variator,Front,04-May,Right-hand drive,Black,2,0
3,3607,806020.0,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966,4.0,Automatic,4x4,04-May,Left wheel,White,0,0
4,11726,404060.0,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901,4.0,Automatic,Front,04-May,Left wheel,Silver,4,0


## 10. Encoding

In [72]:
# Peform endcoding on Doors, Wheel, Drive_wheels, Gear_box_type and
# Leather_interior

# Replace values in the 'Doors' column
# 04-May' is replaced with 1, '02-Mar' is replaced with 2, and '>5'
# is replaced with 3
df['Doors'].replace(['04-May', '02-Mar',">5"],
                        [1, 2, 3], inplace=True)

# Replace values in the 'Wheel' column
# 'Left wheel' is replaced with 1, 'Right-hand drive' is replaced with 2
df['Wheel'].replace(['Left wheel', 'Right-hand drive'],
                        [1, 2], inplace=True)

# Replace values in the 'Drive_wheels' column 'Front' is replaced with 1,
# '4x4' is replaced with 2, 'Rear' is replaced with 3
df['Drive_wheels'].replace(['Front', '4x4',"Rear"],
                        [1, 2, 3], inplace=True)

# Replace values in the 'Gear_box_type' column 'Automatic' is replaced with
# 1, 'Tiptronic' is replaced with 2, 'Manual' is replaced with 3, 'Variator'
# is replaced with 4

df['Gear_box_type'].replace(['Automatic', 'Tiptronic',"Manual","Variator"],
                        [1, 2, 3 , 4], inplace=True)


# Replace values in the 'Leather_interior' column 'Yes' is replaced with 1,
# 'No' is replaced with 2

df['Leather_interior'].replace(['Yes', 'No'],
                        [1, 2], inplace=True)

encoded_cols=['Doors', 'Wheel', 'Drive_wheels', 'Gear_box_type', 'Leather_interior' ]
df[encoded_cols].head(5)

,Doors,Wheel,Drive_wheels,Gear_box_type,Leather_interior
0,1,1,2,1,1
1,1,1,2,2,2
2,1,2,1,4,2
3,1,1,2,1,1
4,1,1,1,1,1


## 11. Scaling

In [74]:
df['Levy']

0        10309090.0
1        10001080.0
2               0.0
3          806020.0
4          404060.0
            ...    
19232           0.0
19233      803010.0
19234      803060.0
19235    10208080.0
19236      705030.0
Name: Levy, Length: 18924, dtype: float64

In [75]:
df['Mileage']

0        186005
1        192000
2        200000
3        168966
4         91901
          ...  
19232    300000
19233    161600
19234    116365
19235     51258
19236    186923
Name: Mileage, Length: 18924, dtype: int32

In [76]:
# Create instance of  MinMaxScaler
from joblib import load
scale = MinMaxScaler()

mileage_scaler = MinMaxScaler()
levy_scaler = MinMaxScaler()    

# Reshape the 'Levy' and 'Mileage' columns to be compatible with the scaler
# X2 = df["Levy"].values.reshape(-1, 1)
# X = df["Mileage"].values.reshape(-1, 1)

# Use Min-Max scaling to scale the 'Mileage' and 'Levy' columns 
mileage_scaler.fit(df[['Mileage']])
levy_scaler.fit(df[['Levy']])

df['Mileage'] = mileage_scaler.transform(df[['Mileage']])
df['Levy'] = levy_scaler.transform(df[['Levy']])

# # Scaled values for 'Mileage' and 'Levy'
# print(scaledX)
# print(scaledX2)

# # Update the 'Mileage' and 'Levy' columns in the DataFrame with the scaled values
# df["Mileage"] = scaledX
# df["Levy"] = scaledX2


df[['Levy','Mileage']].head(5)

dump(mileage_scaler, 'mileage_scaler.joblib')
dump(levy_scaler, 'levy_scaler.joblib')

['levy_scaler.joblib']

In [ ]:
df[['Levy','Mileage']].head(5)

In [ ]:
# print the columns
df.columns

In [ ]:
df.head(50).T

In [77]:
# lets separate num attributes and categorical attributes from data
num_attribs = ['Levy','Prod_year', 'Engine_volume','Doors', 'Mileage', 'Cylinders', 'Airbags']
cat_attribs = ['Manufacturer','Category', 'Leather_interior', 'Fuel_type', 'Gear_box_type', 'Drive_wheels', 'Wheel', 'Color', 'With_Turbo']

In [78]:
# Create separate df for all
num_cars = df[num_attribs]
y = df['Price']
cat_cars = df[cat_attribs]

In [79]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Initialize standar scaler object
scaler = StandardScaler()

# perform scalarization on numerical attributes
num_cars = scaler.fit_transform(num_cars)

In [80]:
from sklearn.pipeline import Pipeline

# created pipeline for scalarization
num_pipeline = Pipeline([
    ('std_scaler', StandardScaler())])

In [92]:
from sklearn.compose import ColumnTransformer

# created pipeline for one hot encoding to transform variables
full_pipeline = ColumnTransformer([
    ('num',num_pipeline, num_attribs),
    ('cat',OneHotEncoder(), cat_attribs)  
])

cars_prepared = full_pipeline.fit_transform(df)
dump(full_pipeline, 'full_pipeline.joblib')

['full_pipeline.joblib']

In [95]:
cars_prepared.shape

(18924, 119)

In [84]:
# perform train test split
X_train, X_test, y_train, y_test = train_test_split(cars_prepared, y, test_size=0.20, random_state = 10)

In [91]:
X_test
df_test = pd.DataFrame(X_test)
print(df_test.head(10).T)

                                                   0  \
0    (0, 0)\t-0.21447004280965076\n  (0, 1)\t-2.2...   

                                                   1  \
0    (0, 0)\t-0.21447004280965076\n  (0, 1)\t-0.8...   

                                                   2  \
0    (0, 0)\t-0.19175335958773612\n  (0, 1)\t0.54...   

                                                   3  \
0    (0, 0)\t-0.14801552118517627\n  (0, 1)\t0.36...   

                                                   4  \
0    (0, 0)\t-0.16240132804598617\n  (0, 1)\t-0.1...   

                                                   5  \
0    (0, 0)\t-0.1628394070614338\n  (0, 1)\t0.015...   

                                                   6  \
0    (0, 0)\t-0.21447004280965076\n  (0, 1)\t-1.9...   

                                                   7  \
0    (0, 0)\t-0.1770010118671998\n  (0, 1)\t0.368...   

                                                   8  \
0    (0, 0)\t0.5235581344824909\n  (0, 1